In [1]:
%load_ext autoreload

In [2]:
%autoreload
from cropcal import *

# Cropland GIS data calibrator
This notebook comprises a method to clibrate the cropland irrigated areas serviced by groundwater resources from the North Western Sahara Aquifer System (NWSAS). The mehtod uses several GIS data layers and national statistical data to accomplish the calibration. The analysis is intended to eliminate four different sources of error/misclassification:
1. Natural vegetation
2. Rain-fed crops
3. Irrigated crops using surface water
4. Irrigated crops using ground water from external sources (in relation to NWSAS) such as shallow aquifers.

## Data loading and preparation
If you already have a cleaned dataset ready for the calibration process, please skip to the [**calibration**](#calibration) section. The first stage in the analysis is the data loading, merging and preparation. For this some procedures are followed next.
### Cleanning of full dataset:
First load the full dataset containing the data for *cropland*, *cropland area*, *irrigated area*, *proximity to dams*, *proximity to rivers*, *administrative boundaries* and *georeferenced coordinates*. Provide the correct `path` for the dataset. The data will be then filtered to only consider data cells that contain croplands. For this, provide the correct names for the `file_name`,`admin_name`,`cropland_name` and `output_path`.

In [6]:
input_path = 'Calibration data/' #the path for the folder containing the dataset
file_name = 'OutputData.gz' #the name of the dataset
admin_name = 'Province' #name of the administrative divition considered in the analysis (e.g. Province, Governorate, Country)
cropland_name = 'ESACropland' #name of the parameter containing the cropland data to calibrate
additional_crop_name = 'ESAFROMCropland'
output_path = 'Calibration data/' #output path for the cleaned dataset

In [7]:
df = pd.read_csv(input_path + file_name)

In [8]:
df.dropna(subset=[admin_name], inplace=True)

In [9]:
clean_data(df, 'ESAFROM_su')

In [10]:
df.rename(index=str, columns={'ESACCI_sum':'ESACropland','ESAFROM_su':'ESAFROMCropland',
                              'ProxToRive':'ProxToRivers','IrrigatedAr':'IrrigatedAreas'},
         inplace=True)
df = df[['X','Y','IrrigatedAreas','ESACropland','ESAFROMCropland','ProxToDams','ProxToRivers','Province','NAME_1']]
df['IrrigatedAreas'].fillna(0, inplace = True)

In [11]:
df.to_csv(output_path + 'CleanData.gz', index = False)

<a id='calibration'></a>
## Calibration process
The calibration process follows different criterias, such as:
1. Non-irrigated croplands: This approach suggests that irrigated areas maps or rainfall data should be used, in order to identify the cropland areas that have less probabiliy of using groundwater for irrigation. These areas should be excluded. 
1. Distance from rivers:
   * Distance from non-perennial rivers: This approach suggests that croplands that are very close to non-perennial river beds are excluded. 
   * Distance from perennial rivers: This approach suggests that croplands that are very close to perennial rivers are excluded.
    Strahler stream order (or equivalent metric) can be used as a river categorization indicator. This will allow for higher control of the distance from rivers calibration criteria.
1. Size/density of crops: This approach suggests that very small crop fields and/or low crop density areas are excluded from the analysis. 
1. Distance to external irrigation sources: This approach suggests that croplands located in proximity to known external irrigation sources are excluded (e.g. water dams).

In principle, other criteria as terrain elevation, soil composition or terrain slope, can be used for the calibration process. It is up to the expert to define which criteria will be most relevant. 

First provide the right names for the `input_path` directory containing the clean dataset `file` name, the `calibration_file` with the regional statistic values of total cropland area withing each administrative region (`admin_name`) and the `calibration_value_name` inside the calibration file.

In [13]:
input_path = 'Calibration data/' #the path for the folder containing the cleaned dataset
output_path = 'Calibrated data/' #output path for the calibrated dataset
file = 'CleanData.gz'
calibration_file = 'CalibrationData.xlsx'
calibration_value_name = 'CalibrationValue'
df = load_data(input_path + file)
calib_data = pd.read_excel(input_path + calibration_file)

**Note:** if you have not previously defined the `admin_name` and `cropland_name` variables please do it in the next block, otherwise you can skip the next block.

In [14]:
admin_name = 'Province' #name of the administrative divition considered in the analysis (e.g. Province, Governorate, Country)
cropland_name = 'ESACropland' #name of the parameter containing the cropland data to calibrate
additional_crop_name = 'ESAFROMCropland' #name of the parameter containing the extra cropland data to add to zones that have less cropland than the one indicated by the calibration value

### Total cropland area without calibration:
First, the total cropland area found in the dataset is computed for future reference. For this, the pixel cell (`pixel_cell`) size (meters) of the data needs to be provided. Moreover, select the desired units for the analysis (i.e. **dunams** or **hectares**).

In [15]:
cell_size = 21.22645334471613765 #cell size of dataset in meters
units = 'hectares'
cell_area = convert_units(units, df, cell_size, cropland_name)
cell_area = convert_units(units, df, cell_size, additional_crop_name)

In [16]:
get_statistics(df, admin_name, cropland_name, calib_data, calibration_value_name, units)

The current total cropland area is: **873756.7** hectares. This represents a **247.6%** difference against the regional statistics provided.

,ESACropland,CalibrationValue,Difference
Province,,,
1.0,24101.816848,28228,-0.146173
6.0,752.607912,0,inf
7.0,4284.053873,0,inf
9.0,157606.188264,5000,30.521238
16.0,61304.161871,3000,19.434721
17.0,7571.757055,0,inf
18.0,82795.300672,60000,0.379922
20.0,23389.161093,30000,-0.220361
22.0,579.840653,4000,-0.855040


### Analytical Hierarchical Process (AHP):
The Analytic Hierarchy Process (AHP), introduced by Thomas Saaty (1980), is an effective tool for
dealing with complex decision making, and may aid the decision maker to set priorities and make
the best decision. It reduces complex decisions to a series of pairwise comparisons, and then
synthesizes the results.

The AHP generates a weight for each evaluation criterion according to the decision maker’s
pairwise comparisons of the criteria. The higher the weight, the more important the corresponding
criterion. Next, for a fixed criterion, the AHP assigns a score to each option according to the
decision maker’s pairwise comparisons of the options based on that criterion. The higher the score,
the better the performance of the option with respect to the considered criterion. Finally, the AHP
combines the criteria weights and the options scores, thus determining a global score for each
option, and a consequent ranking. The global score for a given option is a weighted sum of the
scores it obtained with respect to all the criteria.

To start this process, a pairwise comparison matrix needs to be developed, scoring all the decision criteria selected. An example can be seen next:

|Parameter|Criteria<sub>j (j=1)</sub>|Criteria<sub>j (j=2)</sub>|Criteria<sub>j (j=3)</sub>|Weigths (%)|
|---|---|---|---|---|
|**Criteria<sub>k (k=1)</sub>**|1|1/A|1/B|$\omega_1$|
|**Criteria<sub>k (k=2)</sub>**|A|1|1/C|$\omega_2$|
|**Criteria<sub>k (k=3)</sub>**|B|C|1|$\omega_3$|

The relative scores assigned to each criteria should be on a scale from 1 to 9, following the next definition:

|Values|Interpretation|
|---|---|
|1|k and j are equally important|
|3|k is slightly more important than j|
|5|k is more important than j|
|7|k is strongly more important than j|
|9|k is absolutely more important than j|

An example of such matrix for some posible decision criteria can be seen next:

|Parameter|Distance to Rivers|Distance to Dams|Cropland area|Weigths (%)|
|---|---|---|---|---|
|**Distance to Rivers**|1|3|5|64.79%|
|**Distance to Dams**|1/3|1|2|22.99%|
|**Cropland area**|1/5|1/2|1|12.21%|

In the next block please define the left side of the matrix for the selected criterias. The criteria can be as many as wanted, just add a new line to the matrix with the corresponding scores. Remember to be consistent with the order of the decision criteria in both sides of the matrix (horizontal and vertical). 

Moreover, please define for each selected criteria if it has an `inverted_relation`. For example, take the "Proximity to Rivers" criteria, such criteria has an inverted realation, because if a suposed cropland is closer to the river, it has a high probability of being vegetation on the surroundings of the river. Thus, the lower distance to the river, the larger posibility of it not beign a cropland or to be using surface water for irrigation. This criteria should be marked with a 'y' indicating that it has an inverted relation.

In [17]:
# A matrix of four criteria would look like this:
# criteria_dic = {'ProxToRivers':[1],
#                 'ProxToDams':[1/3,1],
#                 'Cropland':[1/5,1/2,1],
#                 'ProxToWells':[3,5,7,1]}

df['CroplandArea'] = df[cropland_name]

criteria_dic = {'ProxToDams':[1],
                'ProxToRivers':[2,1],
                'CroplandArea':[1/3,1/4,1],
                'IrrigatedAreas':[4,3,9,1]}

inverted_relation = {'ProxToDams':'y', #indicate for each criteria 'y' if it has an inverted relation or 'n' otherwise
                     'ProxToRivers':'y',
                     'CroplandArea':'y',
                     'IrrigatedAreas':'y'}

matrix_scores, criteria = get_matrix(criteria_dic)
w_categories, CR = AHP(matrix_scores)

### Checking the consistency
It is posible to check the consistency of the decision matrix, by calculating the Consistency Ratio (CR).

If CR is lower than 0.1 it can be said that the matrix is consistent. For example:

$$
A=
\left[ {\begin{array}{ccc}
1 & 3 & 1/3\\
1/3 & 1 & 3\\
3 & 1/3 & 1\\
\end{array} } \right]
=> CR = 1.149 => inconsistent
$$

$$
A=
\left[ {\begin{array}{ccc}
1 & 3 & 3\\
1/3 & 1 & 3\\
1/3 & 1/3 & 1\\
\end{array} } \right]
=> CR = 0.118 => slightly~ inconsistent
$$

$$
A=
\left[ {\begin{array}{ccc}
1 & 3 & 5\\
1/3 & 1 & 3\\
1/5 & 1/3 & 1\\
\end{array} } \right]
=> CR = 0.033 => consistent
$$

Check the consistency of the matrix by running the following block:

In [18]:
display_scores(w_categories, CR, criteria)

The computed **CR** for the provided decision matrix is: **0.012**, thus the decision matrix is consistent

The score for criteria **ProxToDams** is: **14.14%**

The score for criteria **ProxToRivers** is: **22.88%**

The score for criteria **CroplandArea** is: **5.56%**

The score for criteria **IrrigatedAreas** is: **57.42%**

### Run the calibration:
Run the next block to find the threshold for all administrative divisions and calibrate the cropland area acording to the required `accuracy` and the calibraion values provided in the `calib_data` file. It may take a while depending on the dataset size.

In [19]:
accuracy = 0.15 #set the desired accuracy/tolerance for the calibration
restrictions = {'ProxToRivers': 0}
run_calibration(df, cropland_name, accuracy, calib_data, w_categories, cell_size, admin_name, calibration_value_name, list(criteria_dic.keys()), inverted_relation, restrictions, additional_crop_name)
get_statistics(df, admin_name, cropland_name, calib_data, calibration_value_name, units)

Progress: [####################] 100.0%


The current total cropland area is: **261967.76** hectares. This represents a **4.2%** difference against the regional statistics provided.

,ESACropland,CalibrationValue,Difference
Province,,,
1.0,24101.816848,28228,-0.146173
6.0,0.000000,0,0.000000
7.0,0.000000,0,0.000000
9.0,4416.574877,5000,-0.116685
16.0,3226.278235,3000,0.075426
17.0,0.000000,0,0.000000
18.0,67801.541469,60000,0.130026
20.0,34489.500320,30000,0.149650
22.0,1770.396215,4000,-0.557401


### Save the calibrated data:
If you are pleased with the calibration process, run the following block to save the data. Remember to set the `output_file` to the desired name for the output data. The calibrated dataset will be saved in the same `input_path` directory defined in the calibration process.

In [20]:
output_file = 'CalibratedData.gz' #define this to the desired file name
create_folder(output_path)
save_data(df, output_path + output_file)